# Global fit of the model on all the lenslets
To obtain a continuous spatial covering of an image, the focal plane is divided into many different cells by the lenslet array. The signal from each pixel is then sent to a spectrograph, so that in the end every pixel generate one spectre, giving one position. Each spectre is a sum of gaussian spots.

The inverse problem approach allows to reduce errors on the raw data, which will in turn allows a better data reduction. We start with a direct model following the equation Mx = y-  , with Mx the unknown model, y the data, . To obtain the projection operator, we inverse the model: instead of having the position x(λ) on the detector, we want the wavelengths λ(x).

Depending on the mode (YJ or YH), a lenslet has 3 or 4 spots. Here, we are initialising the model of these spots.
We need to compute the amplitudes a, full widths at half maximum fwhm and positions of the spots in the lenslets, and adjust the polynomial laws with cx the coefficients on x axis and cy on y axis. 

In [27]:
#Packages
using PIC

using Plots, StatsBase, Statistics, TwoDimensional, OptimPackNextGen, DelimitedFiles
using FITSIO, ProgressMeter
using DataFrames, CSV

plotly()                                 # Backend of Plots package

Plots.PlotlyBackend()

In [14]:
#Wavelengths
#lasers are placed in front of the lenslets, in order to calibrate over different bands.
#lasers are distributed on the detector following the dispersion law.
#3 or 4 lasers are used depending on which mode is used.
λ1 = 987.72e-9                           # laser 1 
λ2 = 1123.71e-9                          # laser 2 
λ3 = 1309.37e-9                          # laser 3
λ4 = 1545.10e-9                          # laser 4  
λlaser = [λ1,λ2,λ3]
nλ = length(λlaser)
λ0 = mean(λlaser);                       # reference
wavelengthrange = LinRange(850e-9,1600e-9,50); # coarse wavelength range of the instrument

## Getting the polynomial coefficients from txt files

x(λ) = cx0 + cx1((λ-λ0)/λ0) + cx2((λ-λ0)/λ0)²

y(λ) = cy0 + cy1((λ-λ0)/λ0) + cy2((λ-λ0)/λ0)²

c0: order 0 coefficient, corresponding to the center of the lenslet, 0 = 1025x1025 pixel, so we have to add 1025 to cx0 and cy0.

λ is in meter here, but in the previous Matlab code, λ was in micrometer. c1 and c2 are then normalised and converted.

In [15]:
coeffx = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_x.txt", header = false)
cx0 = coeffx[:,1] .+ 1025;                # cx0
mcx1 = median(coeffx[:,2])*λ0*1e6;        # median of cx1
mcx2 = median(coeffx[:,3])*(λ0*1e6)^2;    # median of cx2

coeffy = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_y.txt", header = false)
cy0 = coeffy[:,1].+ 1025;                 # cy0
mcy1 = median(coeffy[:,2])*λ0*1e6;        # median of cy1
mcy2 = median(coeffy[:,3])*(λ0*1e6)^2;    # median of cy2

## Statistics of input coefficients

In [36]:
cx1 = coeffx[:,2]*λ0*1e6;
cx2 = coeffx[:,3]*λ0^2*1e12;
cy1 = coeffy[:,2]*λ0*1e6;                 # handle the spectre linearity along y
cy2 = coeffy[:,3]*λ0^2*1e12;

h1 = histogram([cx1 cx2], title = "cx Anthony", bins = LinRange(-20,20,100), 
    label = ["cx1" "cx2"], xlabel = "position", legend = :best)

<!DOCTYPE html>
 
 
 Plots.jl

In [17]:
h2 = histogram([cy1 cy2], title = "cy Anthony", bins = LinRange(-100,130,100), 
    label = ["cy1" "cy2"], xlabel = "position", legend = :best)

<!DOCTYPE html>
 
 
 Plots.jl

In [18]:
h3 = histogram(cy1, title = "cy1 Anthony", bins = LinRange(100,105,100), 
    label = "cy1", legend = :best, color= "red")

h4 = histogram(cy2, title = "cy2 Anthony", bins = LinRange(-100,-25,100), 
    label = "cy2", legend = :best)

plot(h3, h4, layout = (1,2), xlabel = "position")

<!DOCTYPE html>
 
 
 Plots.jl

## Preprocessed data

In [19]:
lampData =  read(FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_spec_corrected.fits")[1]);
laserData =  read(FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits")[1]);
#bad pixel map, 0 or 1 value: used as precision map
badpix = Float64.(read(FITS("/home/user/stage/HR_4796-HD_95086/IFS_BP_corrected.fits")[1])); 

## Constants

In [20]:
lensletnumber= length(cx0) # Total number of lenslet in the image

#Initialisation
ainit = [990. , 690. , 310.];
fwhminit = [2.3, 2.4 , 2.7];

#BoundingBox
largeur = 4;
hauteur = 44;
dxmin = 2;
dxmax = 2;
dymin = 21;
dymax = 18;

In [21]:
#Valid lenslet framed by bbox 
valid = (round.(cx0 .- dxmin).>0) .&  (round.(cx0 .+ dxmax).<2048) .&  
(round.(cy0 .- dymin).>0) .&  (round.(cy0 .+ dymax).<2048);

## Initialisation of the tables of results

In [22]:
lenslettab = Array{Union{LensletModel,Missing}}(missing,lensletnumber);
atab = Array{Union{Float64,Missing}}(missing,3,lensletnumber);
fwhmtab = Array{Union{Float64,Missing}}(missing,3,lensletnumber);
ctab = Array{Union{Float64,Missing}}(missing,2,3,lensletnumber);

## Fit over all the lenslets

In [23]:
#Progress bar of the fit
p = Progress(lensletnumber; showspeed=true)

Threads.@threads for i in findall(valid) # i indice des lenslet valides
    #Initialisation
    bbox = round(Int, BoundingBox(cx0[i,1]-dxmin, cx0[i,1]+dxmax, cy0[i,1]-dymin, cy0[i,1]+dymax));
    lenslettab[i] = LensletModel(λ0,nλ-1, bbox); #Contains the model for each lenslet
    Cinit= [ [cx0[i,1] mcx1 mcx2]; [cy0[i,1] mcy1 mcy2] ];
    xinit = vcat([fwhminit[:],Cinit[:]]...);
    #Optimisation
    laserDataView = view(laserData, bbox);
    badpixview = view(badpix,bbox)
    lkl = LikelihoodIFS(lenslettab[i],λlaser, laserDataView,badpixview);
    cost(x::Vector{Float64}) = lkl(x)
    try #The try/catch statement allows for Exceptions to be tested for
        xopt = vmlmb(cost, xinit; verb=false,ftol = (0.0,1e-8),maxeval=500);
        #Backup of the parameters into vectors
        (fwhmopt,copt) = (xopt[1:(nλ)],reshape(xopt[(nλ+1):(3*nλ)],2,:));
        atab[:,i] = lkl.amplitude;
        fwhmtab[:,i] = fwhmopt
        ctab[:,:,i] = copt
    catch
        continue
    end
    next!(p)
end
ProgressMeter.finish!(p)

Progress: 100%|███████████████████████████| Time: 1:00:44 ( 0.19  s/it)mt)m


## Backup as CVS files

In [28]:
#Tables of results without bad fit, marked as "missing"
atab_valid = collect(skipmissing(atab));
fwhmtab_valid = collect(skipmissing(fwhmtab));
ctab_valid = collect(skipmissing(ctab));

#separation de ctab en cxtab et cytab
cxtab = reshape(ctab_valid[:,1,1],2,: )[1,:,:];
reshape(cxtab,3,:);
cytab = reshape(ctab_valid[:,1,1],2,: )[2,:,:];
reshape(cytab,3,:);

#backup of results in CVS files
CSV.write("atab.csv",  DataFrame(reshape(atab_valid, 3 , : ), :auto));
CSV.write("fwhmtab.csv",  DataFrame(reshape(fwhmtab_valid, 3, :), :auto));
CSV.write("cxtab.csv",  DataFrame(reshape(cxtab,3,:), :auto));
CSV.write("cytab.csv",  DataFrame(reshape(cytab,3,:), :auto));

## Statistics of resulting parameters

In [30]:
#amplitudes
a_csv = CSV.read("atab.csv", DataFrame; datarow = 1, transpose = true);
a0_hist = (a_csv[:,2]);
a1_hist = (a_csv[:,3]);
a2_hist = (a_csv[:,4]);
histogram([a0_hist a1_hist a2_hist], title = "amplitudes", label = ["a0" "a1" "a2"],
    bins = LinRange(-200,1500, 100), xlabel = "position", ylabel = "amplitude")

<!DOCTYPE html>
 
 
 Plots.jl

In [31]:
#fwhm
fwhm_csv = CSV.read("fwhmtab.csv", DataFrame; datarow = 1, transpose=true);
fwhm0_hist = (fwhm_csv[:,2]);
fwhm1_hist = (fwhm_csv[:,3]);
fwhm2_hist = (fwhm_csv[:,4]);
histogram([fwhm0_hist fwhm1_hist fwhm2_hist], title = "fwhm", label = ["fwhm0" "fwhm1" "fwhm2"],
    bins = LinRange(1.5,3.5, 100),  xlabel = "position", ylabel = "fwhm", legend = :best)

<!DOCTYPE html>
 
 
 Plots.jl

In [32]:
#polynomial coefficients cx
cx_csv = CSV.read("cxtab.csv", DataFrame; datarow = 1, transpose=true);
cx1_hist = (cx_csv[:,3]);
cx2_hist = (cx_csv[:,4]);
histogram([cx1_hist cx2_hist], title = "cx", label = ["cx1" "cx2"],
    bins = LinRange(-20,20,100), xlabel = "position", ylabel = "cx", legend = :best)

<!DOCTYPE html>
 
 
 Plots.jl

In [38]:
#polynomial coefficients cy
cy_csv = CSV.read("cytab.csv", DataFrame; datarow = 1, transpose=true);
cy1_hist = (cy_csv[:,3]);
cy2_hist = (cy_csv[:,4]);
histogram([cy1_hist cy2_hist], title = "cy", label = ["cy1" "cy2"],
     bins = LinRange(-100,130,100), xlabel = "position", ylabel = "cy", legend = :best)

<!DOCTYPE html>
 
 
 Plots.jl

In [40]:
#polynomial coefficients cy1 and cy2
h5 = histogram(cy1_hist, title = "cy1", label = "cy1",
     bins = LinRange(100,105,100), xlabel = "position", ylabel = "cy", legend = :best, color = "red");

h6 = histogram(cy2_hist, title = "cy2", label = "cy2",
     bins = LinRange(-100,-25,100), xlabel = "position", ylabel = "cy", legend = :best);

plot(h5, h6, layout = (1,2))

<!DOCTYPE html>
 
 
 Plots.jl

## Identifying and removing bad lenslet
After identifying bad lenslet in the histograms and visualising them with a heatmap, we remove them using mean absolute difference (mad). If the difference is > 5σ (5 pixels), the lenslet is removed.

In [39]:
#Lenslet visualisation
bbox = BoundingBox(xmin=, xmax=, ymin=, ymax=);

x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);

plt = scatter(cx0[1:10,1], cx0[1:10,1])
ldata = view(lampData,bbox);
heatmap(y_axis, x_axis, ldata, title = "Exemple du spectre d'une lenslet au centre")


LoadError: [91msyntax: unexpected ","[39m

In [44]:
@show mad(fwhm0_hist);
@show mad(fwhm1_hist); 
@show mad(fwhm2_hist);
@show mad(cx1_hist);
@show mad(cx2_hist);
@show mad(cy1_hist);
@show mad(cy2_hist);

mad(fwhm0_hist) = 0.08259490727200977
mad(fwhm1_hist) = 0.06390095539514035
mad(fwhm2_hist) = 0.160609110646972
mad(cx1_hist) = 0.595950694545201
mad(cx2_hist) = 3.15366386005774
mad(cy1_hist) = 0.5118303219421795
mad(cy2_hist) = 3.078040545037407
